# <font color='darkred'>Tese de Mestrado</font>
## Tratamento de Dados
- Trabalhadores em apenas 1 empresa;
- Desinflação;
- Tratamento de variáveis e do tipo das mesmas.

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('dataset_final.xlsx')
df.head(2)

In [ ]:
df.shape

#### Considerar Trabalhadores que estão apenas numa 1 empresa

In [ ]:
df_filtrado = df[~df.apply(lambda row: row.astype(str).str.contains('\|').any(), axis=1)]
df_filtrado.shape

Numa amostra inicial de 198 019 trabalhadores, serão considerados 180 720, o que corresponde a aproximadamente 91,24%. Posto isto, **8,76% dos trabalhadores** foram **excluídos** por estarem em mais do que uma empresa ao mesmo tempo.

#### Correção do Tipo de Variáveis
- Existem variáveis como a nuemp que estão com 652540.0 por isso, serão identificados todos estes casos e eliminado o .0.

In [ ]:
int_0 = df_filtrado['nuemp_10'].value_counts()
int_0

In [ ]:
colunas_objeto = df_filtrado.select_dtypes(include=['object']).columns
colunas_interesse = [col for col in colunas_objeto if not col.startswith('caem1l_')]

# Função para remover ".0" de valores numéricos representados como strings
def remover_sufixo_decimal(value):
    if isinstance(value, str) and value.endswith('.0'):
        return value[:-2]  # Remove os últimos dois caracteres (".0")
    return value

for col in colunas_interesse:
    df_filtrado[col] = df_filtrado[col].apply(remover_sufixo_decimal)

In [ ]:
int_0 = df_filtrado['nuemp_10'].value_counts()
int_0

#### Desinflacionar as variáveis "rganho"
- Com vista a garantir uma uniformização dos salários ao longo dos anos, torna-se crucial desinflacionar os ganhos;
- O ano de 2012 será considerado como ano base para todos os cálculos;
- O Índice de Preço do Consumidor será o aplicado em Portugal.

In [ ]:
df_ganho = df_filtrado.copy()

# IPC para cada ano
ipc = {
    2009: 92.574,
    2010: 93.872,
    2011: 97.302,
    2012: 100.000,
    2013: 100.274,
    2014: 99.996,
    2015: 100.483,
    2016: 101.094,
    2017: 102.477,
    2018: 103.496,
    2019: 103.846
}

# Índice base para 2012
base_ipc = ipc[2012]

# Função para desinflacionar
def desinflacionar(valor, year):
    try:
        valor_float = float(valor)
        return round(valor_float * (base_ipc / ipc[year]), 2)
    except ValueError:
        # Devolve o valor original se não for possível converter para float (ex: "Não Trabalhou")
        return valor

# Aplicar a desinflação para cada ano
for year in range(2009, 2020):
    rganho_column = f'rganho_{str(year)[2:]}'
    
    # Verifica se a coluna existe no DataFrame
    if rganho_column in df_ganho.columns:
        df_ganho[rganho_column] = df_ganho[rganho_column].apply(lambda x: desinflacionar(x, year) if pd.notnull(x) else x)

df_ganho.head()

In [ ]:
df_ganho.shape

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Número de NaNs em cada coluna
nans_count = df_ganho.isna().sum()

# Filtrar para mostrar apenas as colunas com valores diferentes de 0
nans_count_non_zero = nans_count[nans_count != 0]
nans_count_non_zero

#### Coluna rganho/total de horas
De forma a comparar o rganho entre trabalhadores, torna-se crucial perceber o rganho por hora trabalhada. 
A primeira condição df_rganho_h[rganho_col] == "Não Trabalhou" verifica se o valor é "Não Trabalhou". Se for, a nova coluna rganho_h_ recebe "Não Trabalhou".
A segunda condição df_rganho_h[rganho_col].isna() verifica se o valor é NaN. Se for, rganho_h_ também será NaN.
Cálculo com Horas Normais (hnormais) e Extras (hextra):

Se hnormais for NaN, rganho_h_ é automaticamente NaN (conforme tua segunda regra).
Se hextra for NaN, o cálculo de rganho_h_ usa apenas hnormais.
Se ambos hnormais e hextra não forem NaN, o cálculo usa a soma de ambos.

In [ ]:
df_rganho_h = df_ganho.copy()
for year in range(9, 20):
    rganho_col = f'rganho_{year:02d}'
    hnormais_col = f'hnormais_{year:02d}'
    hextra_col = f'hextra_{year:02d}'
    rganho_h_col = f'rganho_h_{year:02d}'
    
    # Conversão para numérico das horas normais e extras
    hnormais = pd.to_numeric(df_rganho_h[hnormais_col], errors='coerce')
    hextra = pd.to_numeric(df_rganho_h[hextra_col], errors='coerce')
    rganho = pd.to_numeric(df_rganho_h[rganho_col], errors='coerce')
    
    # Soma de horas normais e extras
    total_horas = hnormais.fillna(0) + hextra.fillna(0)
    
    # Lógica de cálculo
    df_rganho_h[rganho_h_col] = np.where(
        df_rganho_h[rganho_col] == "Não Trabalhou",
        "Não Trabalhou",
        np.where(
            df_rganho_h[rganho_col].isna(),
            df_rganho_h[rganho_col],  # Mantém o valor original de rganho_col se for NaN
            np.where(
                hnormais.isna(),  # Se hnormais for NaN
                df_rganho_h[hnormais_col],  # Mantém o valor original de hnormais_col
                np.where(
                    total_horas == 0,  # Se total de horas for 0, define como 0
                    0,
                    (rganho / total_horas).round(2)  # Caso contrário, faz o cálculo
                )
            )
        )
    )
df_rganho_h.head()

In [ ]:
colunas = df_rganho_h.columns.tolist()
novas_colunas = []

colunas_fixas = ['ntrab', 'sexo', 'ano_nascimento']
novas_colunas.extend(colunas_fixas)

for year in range(9, 20):
    prefixo_colunas = [col for col in colunas if col.endswith(f'_{year:02d}') and not col.startswith('rganho_h')]
    rganho_h_col = f'rganho_h_{year:02d}'
    novas_colunas.extend(prefixo_colunas)
    if rganho_h_col in colunas:
        novas_colunas.append(rganho_h_col)

colunas_restantes = [col for col in colunas if col not in novas_colunas]
novas_colunas.extend(colunas_restantes)

df_rganho_h = df_rganho_h[novas_colunas]
df_rganho_h.head()

In [ ]:
with pd.option_context('display.max_columns', None):
    print(df_rganho_h.columns.tolist())

In [ ]:
df_rganho_h.shape

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Número de NaNs em cada coluna
nans_count = df_rganho_h.isna().sum()

# Filtrar para mostrar apenas as colunas com valores diferentes de 0
nans_count_non_zero = nans_count[nans_count != 0]
nans_count_non_zero

#### Tratamento das variáveis rganho_h_

In [ ]:
rganho_h_cols = [col for col in df_rganho_h.columns if col.startswith('rganho_h_')]

df_ganho_filtrado_temp = df_rganho_h.copy()

for col in rganho_h_cols:
    df_ganho_filtrado_temp[col] = pd.to_numeric(df_ganho_filtrado_temp[col].replace("Não Trabalhou", np.nan), errors='coerce')

# Box Plot
plt.figure(figsize=(15, 8))
df_ganho_filtrado_temp.boxplot(column=rganho_h_cols)
plt.title('Box Plot dos Ganhos por Hora por Ano (Sem Outliers)')
plt.xlabel('Ano')
plt.ylabel('Ganho por Hora')
plt.xticks(rotation=45)
plt.show()

In [ ]:
outliers = {}

for col in rganho_h_cols:
    col_values = pd.to_numeric(df_rganho_h[col].replace("Não Trabalhou", np.nan), errors='coerce')
    
    # Limites de 0,5% inferior e superior
    lower_limit = col_values.quantile(0.005)
    upper_limit = col_values.quantile(0.995)
    
    # Identifica os outliers
    outliers_mask = (col_values < lower_limit) | (col_values > upper_limit)
    outliers[col] = df_rganho_h[outliers_mask]
    num_outliers = outliers[col].shape[0]
    
    print(f"{col}: {num_outliers} trabalhadores com outliers.")

In [ ]:
df_ganho_filtrado_temp = df_rganho_h.copy()

for col in rganho_h_cols:
    df_ganho_filtrado_temp[col] = pd.to_numeric(df_ganho_filtrado_temp[col].replace("Não Trabalhou", np.nan), errors='coerce')


for col in rganho_h_cols:
    col_values = df_ganho_filtrado_temp[col].dropna()
    lower_limit = col_values.quantile(0.005)
    upper_limit = col_values.quantile(0.995)
    
    df_ganho_filtrado_temp[col] = np.where((df_ganho_filtrado_temp[col] < lower_limit) | (df_ganho_filtrado_temp[col] > upper_limit), np.nan, df_ganho_filtrado_temp[col])

# Box Plot
plt.figure(figsize=(15, 8))
df_ganho_filtrado_temp.boxplot(column=rganho_h_cols)
plt.title('Box Plot dos Ganhos por Hora por Ano (Sem Outliers)')
plt.xlabel('Ano')
plt.ylabel('Ganho por Hora')
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_outliers = pd.DataFrame(index=df_rganho_h.index)

for col in rganho_h_cols:
    col_values = pd.to_numeric(df_rganho_h[col].replace("Não Trabalhou", np.nan), errors='coerce')
    
    # Limites de 0,5% inferior e superior
    lower_limit = col_values.quantile(0.005)
    upper_limit = col_values.quantile(0.995)
    
    df_outliers[col] = (col_values < lower_limit) | (col_values > upper_limit)

df_ganho_h_final = df_rganho_h.copy()

df_ganho_h_final = df_ganho_h_final[~df_outliers.any(axis=1)]

# Nº  trabalhadores  eliminados
num_trabalhadores_perdidos = df_rganho_h.shape[0] - df_ganho_h_final.shape[0]

print(f"Foram eliminados {num_trabalhadores_perdidos} trabalhadores por terem pelo menos um outlier.")

In [ ]:
df_ganho_h_final.shape

Após a eliminação dos outliers desta variável, foi **eliminado** cerca de **0,93% do dataframe**.

#### Conversão do CAE para Númerico
- O CAE contém letras de A a U. Para facilitar o desenvolvimento de Modelos, estas colunas serão convertidas para numéricas, sendo que os números serão sequênciais respeitando a mesma ordem das letras.

In [ ]:
df_cae = df_ganho_h_final.copy()

In [ ]:
caem1l_columns = [col for col in df_cae.columns if col.startswith('caem1l_')]
unique_values_set = set()
for col in caem1l_columns:
    unique_values_set.update(df_ganho[col].unique())

unique_values_list = list(unique_values_set)

print(f"Valores únicos nas colunas que começam com 'prof_3d': {unique_values_list}")

In [ ]:
# Conversão de letras para números
mapping = {chr(i): i - 64 for i in range(65, 91)}  # Mapeia 'A' a 'Z' para 1 a 26

for col in caem1l_columns:
    df_cae[col] = df_cae[col].replace(mapping)

In [ ]:
df_cae.head(2)

#### Tratamento da Profissão
- A variável prof_3d contém um valor não numérico R99 (Ignorada) pelo que será substituído por em 9999.

In [ ]:
df_prof = df_cae.copy()

In [ ]:
prof_3d_columns = [col for col in df_prof.columns if col.startswith('prof_3d_')]

unique_values_set = set()
for col in prof_3d_columns:
    unique_values_set.update(df_prof[col].unique())

unique_values_list = list(unique_values_set)

# Verificar se 'R99' está na lista de valores únicos
contains_R99 = 'R99' in unique_values_list
print(f"Contém 'R99': {contains_R99}")

In [ ]:
# Substituir 'R99' por '99' em cada uma das colunas prof_3d
for col in prof_3d_columns:
    df_prof[col] = df_prof[col].replace('R99', '9999')

- A partir do ano de 2010 houve uma resstruturação no armazenamento da informação. As categorias das profissões foram alteradas. Posto isto, consideroou-se crucial encontrar uma correspondência dos valores de 2009 tendo base as categorias definidas em 2010

In [ ]:
df_prof_09_10 = pd.read_excel('prof_09_10.xlsx')
df_prof_09_10.head(2)

In [ ]:
df_prof_09_10.dtypes

In [ ]:
df_prof_09_10['2009'] = pd.to_numeric(df_prof_09_10['2009'], errors='coerce').fillna(0).astype(int)
df_prof_09_10['2010'] = pd.to_numeric(df_prof_09_10['2010'], errors='coerce').fillna(0).astype(int)
df_prof_09_10.dtypes

In [ ]:
df_prof_09_10.head(2)

In [ ]:
mapping_2009_to_2010 = dict(zip(df_prof_09_10['2009'], df_prof_09_10['2010']))

df_prof['prof_3d_09'] = df_prof['prof_3d_09'].map(mapping_2009_to_2010).fillna(df_prof['prof_3d_09'])

In [ ]:
filtered_df = df_prof[df_prof['prof_3d_09'].isin([743, 914, 915])]

count_filtered = filtered_df.shape[0]
print(f"Existem {count_filtered} linhas na coluna 'prof_3d_09' com os valores 743, 914 ou 915.")

In [ ]:
def convert_to_numeric(value):
    try:
        return pd.to_numeric(value, errors='coerce')  # Usa 'coerce' para transformar strings não numéricas em NaN
    except ValueError:
        return value 

# Identifica as colunas de tipo objeto
object_columns = df_prof.select_dtypes(include=['object']).columns

for col in object_columns:
    # Aplica a função de conversão apenas em valores diferentes de "Não Trabalhou"
    df_prof[col] = df_prof[col].apply(lambda x: convert_to_numeric(x) if x != 'Não Trabalhou' else x)

df_prof.head()

#### Exportar o Dataset Final Tratado 

In [ ]:
df_final = df_prof

In [ ]:
df_final.shape

In [ ]:
df_final.to_excel("df_final.xlsx", index=False)

---
Beatriz Lapa - Tese de Mestrado